In [1]:
import torch
from torch import nn

In [2]:
torch.cuda.is_available()


True

In [4]:
device = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print(device)

<class 'torch.cuda.FloatTensor'>


In [4]:
torch.arange(0, 12, 1).type(torch.FloatTensor).cuda()

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       device='cuda:0')

In [5]:
x = torch.arange(0, 1024, 1).type(torch.FloatTensor)
k = torch.tensor([1, 2, -1]).type(torch.FloatTensor)
l = torch.arange(0, 512, 1).type(torch.FloatTensor)
BATCH_SIZE, IN_CH, OUT_CH = 1, 1, 1024

In [6]:
mx = nn.MaxPool1d(kernel_size=2, stride=1, padding=0 ).cuda()

In [6]:
# Pad with len(k)-1 zeros to ensure all non-zero outputs are computed.
h = nn.Conv1d(IN_CH, OUT_CH, kernel_size=512, padding=1, stride=4 ,bias=True)

In [7]:
# Pad with len(k)-1 zeros to ensure all non-zero outputs are computed.
d = nn.Conv2d(IN_CH, OUT_CH, kernel_size=512, padding=511, stride=4 , bias=True)

In [9]:
x.reshape(BATCH_SIZE, IN_CH, -1).size()

torch.Size([1, 1, 1024])

In [8]:
y =d(x.reshape(BATCH_SIZE, IN_CH, -1))
print(y.size())

In [ ]:
mx(y).size()

torch.Size([1, 1024, 383])